# Data Cleaning

## Ali Eddeb

### Sept 06, 2020

Data: ~18,000 jobs scraped from freelancer.com

Goal of this notebook: clean the data

In [1]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import ast

In [2]:
#load in data
df = pd.read_csv('data/2020-09-06_data.csv', header=0)

In [3]:
df.head()

,job_title,description,url,tags,price,avg_bid,bids,days_remaining,verified_payment,promotions,contest
0,I need 20 Typewritters for typing 2190 page in...,I need you to write some articles.,/projects/articles/need-typewritters-for-typin...,"['Article Rewriting', 'Article Writing', 'Ghos...",$15 - $25 / hr,False,0 bids,6 days left,False,[],False
1,College Application Essay,Hello. I'm looking for someone who can write m...,/projects/research-writing/college-application...,"['Academic Writing', 'Creative Writing', 'Essa...",$10,True,2 bids,6 days left,False,[],False
2,Write a Training Website Content ( Read- Give...,"Hi,I am looking for someone to write a trainin...",/projects/content-writing/write-training-websi...,"['Article Rewriting', 'Article Writing', 'Cont...",$10 - $30,False,0 bids,6 days left,True,[],False
3,Develop Excel Dashboard,Looking for professional to Develop Excel Dash...,/projects/excel/develop-excel-dashboard/,"['Excel', 'Excel VBA', 'Microsoft Office']",$56,True,4 bids,6 days left,True,[],False
4,Looking for a Front End Web Developer,Hello Folks!We are looking for a freelance fro...,/projects/html/looking-for-front-end-web-27255...,"['Bootstrap', 'CSS', 'HTML', 'PHP', 'Website D...",$20 - $171,False,0 bids,6 days left,False,[],False


In [4]:
#what are the data types:
df.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 18706 entries, 0 to 18705
Data columns (total 11 columns):
job_title           18706 non-null object
description         18706 non-null object
url                 18706 non-null object
tags                18706 non-null object
price               18706 non-null object
avg_bid             18706 non-null bool
bids                18706 non-null object
days_remaining      18706 non-null object
verified_payment    18706 non-null bool
promotions          18706 non-null object
contest             18706 non-null bool
dtypes: bool(3), object(8)
memory usage: 1.2+ MB


### Actions required for cleaning/processing for each feature:

*Italicized* features indicated cleaning is required.

- job_title: text needs to be vectorized
- description: text needs to be vectorized (let's try doing the job_title first and then explore description)
- url: none
- tags: one hot encode
- *pay*: 
    - if range, 1) convert to pay min and max and 2) convert to numeric (e.g. int)
    - if not, convert to int
- *avg_bid*: convert bolean to int
- *bids*: convert to int
- *days_remaining*: convert to int
- *verified_payment*: convert bolean to int
- promotions: one hot encode
- *contest*: convert bolean to int

In [5]:
#convert boleans to int
df['avg_bid'] = np.where(df['avg_bid'] == True, 1, 0)
df['verified_payment'] = np.where(df['verified_payment'] == True, 1, 0)
df['contest'] = np.where(df['contest'] == True, 1, 0)

In [6]:
df.head()

,job_title,description,url,tags,price,avg_bid,bids,days_remaining,verified_payment,promotions,contest
0,I need 20 Typewritters for typing 2190 page in...,I need you to write some articles.,/projects/articles/need-typewritters-for-typin...,"['Article Rewriting', 'Article Writing', 'Ghos...",$15 - $25 / hr,0,0 bids,6 days left,0,[],0
1,College Application Essay,Hello. I'm looking for someone who can write m...,/projects/research-writing/college-application...,"['Academic Writing', 'Creative Writing', 'Essa...",$10,1,2 bids,6 days left,0,[],0
2,Write a Training Website Content ( Read- Give...,"Hi,I am looking for someone to write a trainin...",/projects/content-writing/write-training-websi...,"['Article Rewriting', 'Article Writing', 'Cont...",$10 - $30,0,0 bids,6 days left,1,[],0
3,Develop Excel Dashboard,Looking for professional to Develop Excel Dash...,/projects/excel/develop-excel-dashboard/,"['Excel', 'Excel VBA', 'Microsoft Office']",$56,1,4 bids,6 days left,1,[],0
4,Looking for a Front End Web Developer,Hello Folks!We are looking for a freelance fro...,/projects/html/looking-for-front-end-web-27255...,"['Bootstrap', 'CSS', 'HTML', 'PHP', 'Website D...",$20 - $171,0,0 bids,6 days left,0,[],0


In [7]:
#convert bids to int
#this involves removing the words 'bids' or 'entries'
#example:
df.loc[0,'bids'].replace('bids','').strip()

&#39;0&#39;

In [8]:
df['bids'] = df['bids'].str.replace('bids','').str.replace('entries','').str.strip().astype(int)

In [9]:
#lets explore 'days_remaining' column
df['days_remaining'].unique()

array([&#39;6 days left&#39;, &#39;4 days left&#39;, &#39;2 days left&#39;, &#39;9 days left&#39;,
       &#39;14 days left&#39;, &#39;11 days left&#39;, &#39;29 days left&#39;, &#39;3 days left&#39;,
       &#39;13 days left&#39;, &#39;19 days left&#39;, &#39;20 hours left&#39;, &#39;19 hours left&#39;,
       &#39;7 days left&#39;, &#39;16 hours left&#39;, &#39;27 days left&#39;, &#39;5 days left&#39;,
       &#39;13 hours left&#39;, &#39;12 hours left&#39;, &#39;11 hours left&#39;, &#39;8 days left&#39;,
       &#39;9 hours left&#39;, &#39;20 days left&#39;, &#39;24 days left&#39;, &#39;5 hours left&#39;,
       &#39;2 hours left&#39;, &#39;28 days left&#39;, &#39;1 day left&#39;, &#39;705 days left&#39;,
       &#39;Ended&#39;, &#39;18 days left&#39;, &#39;10 days left&#39;, &#39;12 days left&#39;,
       &#39;16 days left&#39;, &#39;23 hours left&#39;, &#39;21 hours left&#39;, &#39;18 hours left&#39;,
       &#39;17 hours left&#39;, &#39;17 days left&#39;, &#39;15 hours left&#39;,

There are both hours and days remaining. So what I can do is save it in terms of days remaining and hours will be converted a fraction of a day. Jobs that have ended will be defined as 0.

In [10]:
#Creating masks (True/False) for the 3 scenarios

#CASE 1: DAYS
days_mask = df['days_remaining'].str.contains('day')
#CASE 2: HOURS
hours_mask = df['days_remaining'].str.contains('hour')
#CASE 3: ENDED
end_mask = df['days_remaining'].str.contains('End')

In [11]:
#CASE 1: DAYS
df.loc[days_mask,'days_remaining'] = df.loc[days_mask,'days_remaining'].str.replace(' days? left','',regex=True)
df.loc[days_mask,'days_remaining']

0        6
1        6
2        6
3        6
4        6
        ..
18701    6
18702    6
18703    6
18704    6
18705    6
Name: days_remaining, Length: 16576, dtype: object

In [12]:
#CASE 2: HOURS
df.loc[hours_mask,'days_remaining'] = df.loc[hours_mask,'days_remaining'].str.replace(' hours? left','',regex=True).apply(lambda x : round(int(x)/24,3))
df.loc[hours_mask,'days_remaining']

363      0.833
410      0.792
609      0.667
909      0.542
1059       0.5
         ...  
18445    0.667
18449      0.5
18457    0.042
18459    0.042
18613    0.375
Name: days_remaining, Length: 1932, dtype: object

In [13]:
#CASE 3: Job ENDED or ENDING
df.loc[end_mask,'days_remaining'] = df.loc[end_mask,'days_remaining'].apply(lambda x : 0)
df.loc[end_mask,'days_remaining']

3188     0
3429     0
6224     0
8223     0
8426     0
        ..
18570    0
18605    0
18611    0
18612    0
18658    0
Name: days_remaining, Length: 198, dtype: object

In [14]:
#Finally, convert to float
df['days_remaining'] = df['days_remaining'].astype(float)

In [15]:
df.head()

,job_title,description,url,tags,price,avg_bid,bids,days_remaining,verified_payment,promotions,contest
0,I need 20 Typewritters for typing 2190 page in...,I need you to write some articles.,/projects/articles/need-typewritters-for-typin...,"['Article Rewriting', 'Article Writing', 'Ghos...",$15 - $25 / hr,0,0,6.0,0,[],0
1,College Application Essay,Hello. I'm looking for someone who can write m...,/projects/research-writing/college-application...,"['Academic Writing', 'Creative Writing', 'Essa...",$10,1,2,6.0,0,[],0
2,Write a Training Website Content ( Read- Give...,"Hi,I am looking for someone to write a trainin...",/projects/content-writing/write-training-websi...,"['Article Rewriting', 'Article Writing', 'Cont...",$10 - $30,0,0,6.0,1,[],0
3,Develop Excel Dashboard,Looking for professional to Develop Excel Dash...,/projects/excel/develop-excel-dashboard/,"['Excel', 'Excel VBA', 'Microsoft Office']",$56,1,4,6.0,1,[],0
4,Looking for a Front End Web Developer,Hello Folks!We are looking for a freelance fro...,/projects/html/looking-for-front-end-web-27255...,"['Bootstrap', 'CSS', 'HTML', 'PHP', 'Website D...",$20 - $171,0,0,6.0,0,[],0


Now let's deal with the **PAY** which can be:
1) a pay range
2) an average bid

Let's see if there is any unexpected cases (ie. avg bid with a range)

In [16]:
df[(df['avg_bid'] == 1) & (df['price'].str.contains('-'))]

,job_title,description,url,tags,price,avg_bid,bids,days_remaining,verified_payment,promotions,contest


Good, no unexpected cases.

Cases:
- if range, 1) convert to pay min and max and 2) convert to numeric (e.g. int)
- if not, convert to int

In [17]:
#CASE 1: Convert range to min max

#remove 'hr'
df['price'] = df['price'].str.replace(' / hr','')
df['price']

0         $15 - $25
1               $10
2         $10 - $30
3               $56
4        $20 - $171
            ...    
18701    $20 - $168
18702      $7 - $58
18703    $20 - $168
18704    $20 - $167
18705    $20 - $168
Name: price, Length: 18706, dtype: object

In [19]:
#now create pay max and min 
df['price_min'] = np.NaN
df['price_max'] = np.NaN

In [18]:
#get index of pays that are ranges
range_mask = df['price'][df['avg_bid'] == 0]
range_mask

0          $15 - $25
2          $10 - $30
4         $20 - $171
7        $171 - $512
8          $10 - $30
            ...     
18701     $20 - $168
18702       $7 - $58
18703     $20 - $168
18704     $20 - $167
18705     $20 - $168
Name: price, Length: 1940, dtype: object

In [21]:
df.loc[range_mask.index, 'price'].str.split()

0          [$15, -, $25]
2          [$10, -, $30]
4         [$20, -, $171]
7        [$171, -, $512]
8          [$10, -, $30]
              ...       
18701     [$20, -, $168]
18702       [$7, -, $58]
18703     [$20, -, $168]
18704     [$20, -, $167]
18705     [$20, -, $168]
Name: price, Length: 1940, dtype: object

In [23]:
#populate the min (first index) and max (last index) pays for pay ranges
df.loc[range_mask.index, 'price_min'] = df.loc[range_mask.index, 'price'].str.split().str[0]
df.loc[range_mask.index, 'price_max'] = df.loc[range_mask.index, 'price'].str.split().str[-1]

In [24]:
#Remove '$' sign for pay, pay_min and pay_max
for feature in ['price','price_min','price_max']:
    df[feature] = df[feature].str.replace('$','')

In [25]:
#remove the pay ranges and replace them with NaNs
df.loc[df['avg_bid']==0,'price'] = np.NaN
df.loc[df['avg_bid']==0,'price']

0        NaN
2        NaN
4        NaN
7        NaN
8        NaN
        ... 
18701    NaN
18702    NaN
18703    NaN
18704    NaN
18705    NaN
Name: price, Length: 1940, dtype: object

In [26]:
#convert 'min' to 0
df['price_min'] = df['price_min'].str.replace('min','0')

In [27]:
df.head()

,job_title,description,url,tags,price,avg_bid,bids,days_remaining,verified_payment,promotions,contest,price_min,price_max
0,I need 20 Typewritters for typing 2190 page in...,I need you to write some articles.,/projects/articles/need-typewritters-for-typin...,"['Article Rewriting', 'Article Writing', 'Ghos...",NaN,0,0,6.0,0,[],0,15,25
1,College Application Essay,Hello. I'm looking for someone who can write m...,/projects/research-writing/college-application...,"['Academic Writing', 'Creative Writing', 'Essa...",10,1,2,6.0,0,[],0,NaN,NaN
2,Write a Training Website Content ( Read- Give...,"Hi,I am looking for someone to write a trainin...",/projects/content-writing/write-training-websi...,"['Article Rewriting', 'Article Writing', 'Cont...",NaN,0,0,6.0,1,[],0,10,30
3,Develop Excel Dashboard,Looking for professional to Develop Excel Dash...,/projects/excel/develop-excel-dashboard/,"['Excel', 'Excel VBA', 'Microsoft Office']",56,1,4,6.0,1,[],0,NaN,NaN
4,Looking for a Front End Web Developer,Hello Folks!We are looking for a freelance fro...,/projects/html/looking-for-front-end-web-27255...,"['Bootstrap', 'CSS', 'HTML', 'PHP', 'Website D...",NaN,0,0,6.0,0,[],0,20,171


In [28]:
df.dtypes

job_title            object
description          object
url                  object
tags                 object
price                object
avg_bid               int32
bids                  int32
days_remaining      float64
verified_payment      int32
promotions           object
contest               int32
price_min            object
price_max            object
dtype: object

The 3 pay columns need to be converted to a numeric format.

In [29]:
#Convert pay, pay_min and pay_max to numeric (need to use float because of presence of NaNs)
for feature in ['price','price_min','price_max']:
    df[feature] = df[feature].astype(float)

#### Check for duplicates and NaNs

In [32]:
#checking for NaNsy og
#only acceptable NaNs are in the price, price_min and price_max columns
df.isna().sum()

job_title               0
description             0
url                     0
tags                    0
price                1940
avg_bid                 0
bids                    0
days_remaining          0
verified_payment        0
promotions              0
contest                 0
price_min           16766
price_max           16766
dtype: int64

Good. No NaNs in any of the other columns. Now, let's check duplicates.

In [35]:
#duplicates?
df.duplicated().sum()

543

There are 543 duplicated rows. Let's quickly take a look at a sample of these rows.

In [36]:
df[df.duplicated()]

,job_title,description,url,tags,price,avg_bid,bids,days_remaining,verified_payment,promotions,contest,price_min,price_max
51,FREELANCER TELESALES -- 2,* DEDICATED TELESALES EXECUTIVE REQUIRED FOR B...,/projects/internet-marketing/freelancer-telesa...,"['Internet Marketing', 'Leads', 'Marketing', '...",250.0,1,1,6.0,1,[],0,NaN,NaN
52,Update Angular (TS) Firebase website,Please fully read the task and bid appropriate...,/projects/angular-js/update-angular-firebase-w...,"['AngularJS', 'Google Firebase', 'JavaScript',...",NaN,0,0,6.0,1,[],0,10.0,30.0
55,I need a chatbot website,I want some one to create a chat bot site for ...,/projects/php/need-chatbot-website/,"['Graphic Design', 'JavaScript', 'Natural Lang...",3018.0,1,4,6.0,1,[],0,NaN,NaN
199,Simple Github Tutorial for 1 Program,I'm in need of a simple guide to installing 1 ...,/projects/articles/simple-github-tutorial-for-...,"['Article Writing', 'Coding', 'GitHub', 'Rust'...",23.0,1,5,6.0,0,[],0,NaN,NaN
349,I-deask Mobile Phone App,I need both an android & apple app. I would li...,/projects/android/deask-mobile-phone-app/,"['Android', 'Mobile App Development']",118.0,1,3,6.0,1,[],0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18478,SAS Expert Required -- 4,Details will be shared with winning bidder. I ...,/projects/excel/sas-expert-required-26860600/,"['Excel', 'SAS', 'SQL', 'Statistics']",NaN,0,55,4.0,0,['Sealed'],0,7.0,58.0
18479,Macro and sql expert..,i need the Macro and sql expert for multiple j...,/projects/sql/macro-sql-expert-26855890/,"['Excel Macros', 'Microsoft SQL Server', 'MySQ...",NaN,0,30,4.0,0,['Sealed'],0,27.0,67.0
18480,Macro and sql expert,i need the Macro and sql expert for multiple j...,/projects/sql/macro-sql-expert/,"['Excel Macros', 'Microsoft SQL Server', 'MySQ...",NaN,0,16,4.0,0,['Sealed'],0,15.0,59.0
18481,BUILD ME A LOGO,"D&M FOODS NEED A LOGO FOR SNACKS(CHIPS,KURKURE...",/contest/build-me-a-logo-1802040.html,"['Corporate Identity', 'Graphic Design', 'Illu...",NaN,0,75,14.0,0,['Guaranteed'],1,17.0,17.0


Let's take a look at a couple of examples of duplicated rows:

In [37]:
df[df['job_title'] == 'FREELANCER TELESALES -- 2']

,job_title,description,url,tags,price,avg_bid,bids,days_remaining,verified_payment,promotions,contest,price_min,price_max
44,FREELANCER TELESALES -- 2,* DEDICATED TELESALES EXECUTIVE REQUIRED FOR B...,/projects/internet-marketing/freelancer-telesa...,"['Internet Marketing', 'Leads', 'Marketing', '...",250.0,1,1,6.0,1,[],0,NaN,NaN
51,FREELANCER TELESALES -- 2,* DEDICATED TELESALES EXECUTIVE REQUIRED FOR B...,/projects/internet-marketing/freelancer-telesa...,"['Internet Marketing', 'Leads', 'Marketing', '...",250.0,1,1,6.0,1,[],0,NaN,NaN


In [38]:
df[df['job_title'] == 'I-deask Mobile Phone App']

,job_title,description,url,tags,price,avg_bid,bids,days_remaining,verified_payment,promotions,contest,price_min,price_max
348,I-deask Mobile Phone App,I need both an android & apple app. I would li...,/projects/android/deask-mobile-phone-app/,"['Android', 'Mobile App Development']",118.0,1,3,6.0,1,[],0,NaN,NaN
349,I-deask Mobile Phone App,I need both an android & apple app. I would li...,/projects/android/deask-mobile-phone-app/,"['Android', 'Mobile App Development']",118.0,1,3,6.0,1,[],0,NaN,NaN


In [39]:
df[df['job_title'] == 'Macro and sql expert']

,job_title,description,url,tags,price,avg_bid,bids,days_remaining,verified_payment,promotions,contest,price_min,price_max
18472,Macro and sql expert,i need the Macro and sql expert for multiple j...,/projects/sql/macro-sql-expert/,"['Excel Macros', 'Microsoft SQL Server', 'MySQ...",NaN,0,16,4.0,0,['Sealed'],0,15.0,59.0
18480,Macro and sql expert,i need the Macro and sql expert for multiple j...,/projects/sql/macro-sql-expert/,"['Excel Macros', 'Microsoft SQL Server', 'MySQ...",NaN,0,16,4.0,0,['Sealed'],0,15.0,59.0


Unclear why these duplicates arised but they need to be removed due to redundancy.

In [40]:
df.shape

(18706, 13)

In [41]:
df.drop_duplicates(inplace=True)

In [42]:
df.shape

(18163, 13)

All the cleaning steps will be functionalized and saved in a script for use in this project.